In [ ]:
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import tensorflow as tf


def margin_loss(labels, raw_logits, margin=0.4, downweight=0.5):
    """Penalizes deviations from margin for each logit.
    Each wrong logit costs its distance to margin. For negative logits margin is
    0.1 and for positives it is 0.9. First subtract 0.5 from all logits. Now
    margin is 0.4 from each side.
    Args:
    labels: tensor, one hot encoding of ground truth.
    raw_logits: tensor, model predictions in range [0, 1]
    margin: scalar, the margin after subtracting 0.5 from raw_logits.
    downweight: scalar, the factor for negative cost.
    Returns:
    A tensor with cost for each data point of shape [batch_size].
    """
    logits = raw_logits - 0.5
    positive_cost = labels * tf.cast(tf.less(logits, margin),
                                     tf.float32) * tf.pow(logits - margin, 2)
    negative_cost = (1 - labels) * tf.cast(
        tf.greater(logits, -margin), tf.float32) * tf.pow(logits + margin, 2)
    return 0.5 * positive_cost + downweight * 0.5 * negative_cost


def createVocabulary(input_path, output_path, pad=True, unk=True):
    if not isinstance(input_path, str):
        raise TypeError('input_path should be string')

    if not isinstance(output_path, str):
        raise TypeError('output_path should be string')

    vocab = {}
    with open(input_path, 'r') as fd, \
            open(output_path, 'w+') as out:
        for line in fd:
            line = line.rstrip('\r\n')
            words = line.split()

            for w in words:
                if w == '_UNK':
                    break
                if str.isdigit(w) == True:
                    w = '0'
                if w in vocab:
                    vocab[w] += 1
                else:
                    vocab[w] = 1
        init_vocab = []
        if pad:
            init_vocab.append('_PAD')
        if unk:
            init_vocab.append('_UNK')
        vocab = sorted(vocab, key=vocab.get, reverse=True) + init_vocab

        for v in vocab:
            out.write(v + '\n')


def loadVocabulary(path):
    if not isinstance(path, str):
        raise TypeError('path should be a string')

    vocab = []
    rev = []
    with open(path) as fd:
        for line in fd:
            line = line.rstrip('\r\n')
            rev.append(line)
        vocab = dict([(x, y) for (y, x) in enumerate(rev)])

    return {'vocab': vocab, 'rev': rev}


def sentenceToIds(data, vocab, unk):
    if not isinstance(vocab, dict):
        raise TypeError('vocab should be a dict that contains vocab and rev')
    vocab = vocab['vocab']
    if isinstance(data, str):
        words = data.split()
    elif isinstance(data, list):
        words = data
    else:
        raise TypeError('data should be a string or a list contains words')

    ids = []
    if unk:
        for w in words:
            if str.isdigit(w) == True:
                w = '0'
            ids.append(vocab.get(w, vocab['_UNK']))
    else:
        for w in words:
            if str.isdigit(w) == True:
                w = '0'
            ids.append(vocab.get(w))

    return ids


def padSentence(s, max_length, vocab):
    return s + [vocab['vocab']['_PAD']] * (max_length - len(s))


# compute f1 score is modified from conlleval.pl
def __startOfChunk(prevTag, tag, prevTagType, tagType, chunkStart=False):
    if prevTag == 'B' and tag == 'B':
        chunkStart = True
    if prevTag == 'I' and tag == 'B':
        chunkStart = True
    if prevTag == 'O' and tag == 'B':
        chunkStart = True
    if prevTag == 'O' and tag == 'I':
        chunkStart = True

    if prevTag == 'E' and tag == 'E':
        chunkStart = True
    if prevTag == 'E' and tag == 'I':
        chunkStart = True
    if prevTag == 'O' and tag == 'E':
        chunkStart = True
    if prevTag == 'O' and tag == 'I':
        chunkStart = True

    if tag != 'O' and tag != '.' and prevTagType != tagType:
        chunkStart = True
    return chunkStart


def __endOfChunk(prevTag, tag, prevTagType, tagType, chunkEnd=False):
    if prevTag == 'B' and tag == 'B':
        chunkEnd = True
    if prevTag == 'B' and tag == 'O':
        chunkEnd = True
    if prevTag == 'I' and tag == 'B':
        chunkEnd = True
    if prevTag == 'I' and tag == 'O':
        chunkEnd = True

    if prevTag == 'E' and tag == 'E':
        chunkEnd = True
    if prevTag == 'E' and tag == 'I':
        chunkEnd = True
    if prevTag == 'E' and tag == 'O':
        chunkEnd = True
    if prevTag == 'I' and tag == 'O':
        chunkEnd = True

    if prevTag != 'O' and prevTag != '.' and prevTagType != tagType:
        chunkEnd = True
    return chunkEnd


def __splitTagType(tag):
    s = tag.split('-')
    if len(s) > 2 or len(s) == 0:
        raise ValueError('tag format wrong. it must be B-xxx.xxx')
    if len(s) == 1:
        tag = s[0]
        tagType = ""
    else:
        tag = s[0]
        tagType = s[1]
    return tag, tagType


def computeF1Score(correct_slots, pred_slots):
    correctChunk = {}
    correctChunkCnt = 0
    foundCorrect = {}
    foundCorrectCnt = 0
    foundPred = {}
    foundPredCnt = 0
    correctTags = 0
    tokenCount = 0
    for correct_slot, pred_slot in zip(correct_slots, pred_slots):
        inCorrect = False
        lastCorrectTag = 'O'
        lastCorrectType = ''
        lastPredTag = 'O'
        lastPredType = ''
        for c, p in zip(correct_slot, pred_slot):
            correctTag, correctType = __splitTagType(c)
            predTag, predType = __splitTagType(p)

            if inCorrect == True:
                if __endOfChunk(lastCorrectTag, correctTag, lastCorrectType, correctType) == True and \
                        __endOfChunk(lastPredTag, predTag, lastPredType, predType) == True and \
                        (lastCorrectType == lastPredType):
                    inCorrect = False
                    correctChunkCnt += 1
                    if lastCorrectType in correctChunk:
                        correctChunk[lastCorrectType] += 1
                    else:
                        correctChunk[lastCorrectType] = 1
                elif __endOfChunk(lastCorrectTag, correctTag, lastCorrectType, correctType) != \
                        __endOfChunk(lastPredTag, predTag, lastPredType, predType) or \
                        (correctType != predType):
                    inCorrect = False

            if __startOfChunk(lastCorrectTag, correctTag, lastCorrectType, correctType) == True and \
                    __startOfChunk(lastPredTag, predTag, lastPredType, predType) == True and \
                    (correctType == predType):
                inCorrect = True

            if __startOfChunk(lastCorrectTag, correctTag, lastCorrectType, correctType) == True:
                foundCorrectCnt += 1
                if correctType in foundCorrect:
                    foundCorrect[correctType] += 1
                else:
                    foundCorrect[correctType] = 1

            if __startOfChunk(lastPredTag, predTag, lastPredType, predType) == True:
                foundPredCnt += 1
                if predType in foundPred:
                    foundPred[predType] += 1
                else:
                    foundPred[predType] = 1

            if correctTag == predTag and correctType == predType:
                correctTags += 1

            tokenCount += 1

            lastCorrectTag = correctTag
            lastCorrectType = correctType
            lastPredTag = predTag
            lastPredType = predType

        if inCorrect == True:
            correctChunkCnt += 1
            if lastCorrectType in correctChunk:
                correctChunk[lastCorrectType] += 1
            else:
                correctChunk[lastCorrectType] = 1

    if foundPredCnt > 0:
        precision = 100 * correctChunkCnt / foundPredCnt
    else:
        precision = 0

    if foundCorrectCnt > 0:
        recall = 100 * correctChunkCnt / foundCorrectCnt
    else:
        recall = 0

    if (precision + recall) > 0:
        f1 = (2 * precision * recall) / (precision + recall)
    else:
        f1 = 0

    return f1, precision, recall


class DataProcessor(object):
    def __init__(self, in_path, slot_path, intent_path, in_vocab, slot_vocab, intent_vocab, shuffle=False):
        self.__fd_in = open(in_path, 'r').readlines()
        self.__fd_slot = open(slot_path, 'r').readlines()
        self.__fd_intent = open(intent_path, 'r').readlines()
        if shuffle:
            self.shuffle()
        self.__in_vocab = in_vocab
        self.__slot_vocab = slot_vocab
        self.__intent_vocab = intent_vocab
        self.end = 0

    def close(self):
        self.__fd_in.close()
        self.__fd_slot.close()
        self.__fd_intent.close()

    def shuffle(self):
        from sklearn.utils import shuffle
        self.__fd_in, self.__fd_slot, self.__fd_intent = shuffle(self.__fd_in, self.__fd_slot, self.__fd_intent)

    def get_batch(self, batch_size):
        in_data = []
        slot_data = []
        slot_weight = []
        length = []
        intents = []

        batch_in = []
        batch_slot = []
        max_len = 0

        in_seq = []
        slot_seq = []
        intent_seq = []
        for i in range(batch_size):
            try:
                inp = self.__fd_in.pop()
            except IndexError:
                self.end = 1
                break
            slot = self.__fd_slot.pop()
            intent = self.__fd_intent.pop()
            inp = inp.rstrip()
            slot = slot.rstrip()
            intent = intent.rstrip()

            in_seq.append(inp)
            slot_seq.append(slot)
            intent_seq.append(intent)

            iii = inp
            sss = slot
            inp = sentenceToIds(inp, self.__in_vocab, unk=True)
            slot = sentenceToIds(slot, self.__slot_vocab, unk=True)
            intent = sentenceToIds(intent, self.__intent_vocab, unk=False)
            if None not in intent:
                batch_in.append(np.array(inp))
                batch_slot.append(np.array(slot))
                length.append(len(inp))
                intents.append(intent[0])
            if len(inp) != len(slot):
                print(iii, sss)
                print(inp, slot)
                exit(0)
            if len(inp) > max_len:
                max_len = len(inp)

        length = np.array(length)
        intents = np.array(intents)
        for i, s in zip(batch_in, batch_slot):
            in_data.append(padSentence(list(i), max_len, self.__in_vocab))
            slot_data.append(padSentence(list(s), max_len, self.__slot_vocab))

        in_data = np.array(in_data)
        slot_data = np.array(slot_data)

        for s in slot_data:
            weight = np.not_equal(s, np.full(s.shape, self._DataProcessor__slot_vocab['vocab']['_PAD']))
            weight = weight.astype(np.float32)
            slot_weight.append(weight)
        slot_weight = np.array(slot_weight)
        return in_data, slot_data, slot_weight, length, intents, in_seq, slot_seq, intent_seq

In [ ]:
import os
import argparse
import logging
import sys
import tensorflow as tf
import numpy as np
from tensorflow.contrib.rnn.python.ops import core_rnn_cell
from tensorflow.python.ops import rnn_cell_impl

#from utils import createVocabulary, loadVocabulary, computeF1Score, DataProcessor

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# tf.set_random_seed(20181226)  
# np.random.seed(20181226)
# todo: 1. word pre-train embedding, gru, crf, lr decay

parser = argparse.ArgumentParser(allow_abbrev=False)

# Network
parser.add_argument("--num_units", type=int, default=64, help="Network size.", dest='layer_size',required=False)
parser.add_argument("--model_type", type=str, default='full', help="""full(default) | intent_only
                                                                    full: full attention model
                                                                    intent_only: intent attention model""",required=False)
parser.add_argument("--priority_order", type=str, default='slot_first', help="""Type 'slot_first' or 'intent_first'
                                                                              to decide whose influence ought to calculate first use.""",required=False)
parser.add_argument("--use_crf", type=bool, default=True, help="""use crf for seq labeling""",required=False)
parser.add_argument("--use_embedding", type=str, default='1', help="""use pre-trained embedding""",required=False)
parser.add_argument("--cell", type=str, default='lstm', help="""rnn cell""",required=False) 
parser.add_argument("--iteration_num", type=int, default=1, help="""the number of iteration times""",required=False)

# Training Environment
parser.add_argument("--batch_size", type=int, default=16, help="Batch size.",required=False)
parser.add_argument("--batch_size_add", type=int, default=4, help="Batch size add.",required=False)
parser.add_argument("--max_epochs", type=int, default=100, help="Max epochs to train.",required=False)  
parser.add_argument("--no_early_stop", action='store_false', dest='early_stop',
                    help="Disable early stop, which is based on sentence level accuracy.",required=False)
parser.add_argument("--patience", type=int, default=15, help="Patience to wait before stop.",required=False)
# learn rate param
parser.add_argument("--learning_rate_decay", type=str, default='1', help="learning_rate_decay",required=False)
parser.add_argument("--learning_rate", type=float, default=0.001, help="The initial learning rate.",required=False)
parser.add_argument("--decay_steps", type=int, default=280 * 4, help="decay_steps.",required=False)
parser.add_argument("--decay_rate", type=float, default=0.9, help="decay_rate.",required=False)

# Model and Vocab
parser.add_argument("--dataset", type=str, default='snips', help="""Type 'atis' or 'snips' to use dataset provided by us or enter what ever you named your own dataset.
                Note, if you don't want to use this part, enter --dataset=''. It can not be None""",required=False)
parser.add_argument("--model_path", type=str, default='./model', help="Path to save model.",required=False)
parser.add_argument("--vocab_path", type=str, default='./vocab', help="Path to vocabulary files.",required=False)

# Data
parser.add_argument("--train_data_path", type=str, default='train', help="Path to training data files.",required=False)
parser.add_argument("--test_data_path", type=str, default='test', help="Path to testing data files.",required=False)
parser.add_argument("--valid_data_path", type=str, default='valid', help="Path to validation data files.",required=False)
parser.add_argument("--input_file", type=str, default='seq.in', help="Input file name.",required=False)
parser.add_argument("--slot_file", type=str, default='seq.out', help="Slot file name.",required=False)
parser.add_argument("--intent_file", type=str, default='label', help="Intent file name.",required=False)
parser.add_argument("--embedding_path", type=str, default='', help="embedding array's path.",required=False)

arg = parser.parse_args(''.split())
if arg.dataset=='atis':
    arg.model_type='intent_only'
else:
    arg.model_type='full'

# Print arguments
for k, v in sorted(vars(arg).items()):
    print(k, '=', v)
print()
# use full attention or intent only
if arg.model_type == 'full':
    remove_slot_attn = False
elif arg.model_type == 'intent_only':
    remove_slot_attn = True
else:
    print('unknown model type!')
    exit(1)

# full path to data will be: ./data + dataset + train/test/valid
if arg.dataset == None:
    print('name of dataset can not be None')
    exit(1)
elif arg.dataset == 'snips':
    print('use snips dataset')
elif arg.dataset == 'atis':
    print('use atis dataset')
else:
    print('use own dataset: ', arg.dataset)
full_train_path = os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/data', arg.dataset, arg.train_data_path)
full_test_path = os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/data', arg.dataset, arg.test_data_path)
full_valid_path = os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/data', arg.dataset, arg.valid_data_path)

createVocabulary(os.path.join(full_train_path, arg.input_file), os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'in_vocab'))
createVocabulary(os.path.join(full_train_path, arg.slot_file), os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'slot_vocab'))
createVocabulary(os.path.join(full_train_path, arg.intent_file), os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'intent_vocab'))
# return map: {'vocab': vocab, 'rev': rev}, vocab: map, rev: array
in_vocab = loadVocabulary(os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'in_vocab'))
slot_vocab = loadVocabulary(os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'slot_vocab'))
intent_vocab = loadVocabulary(os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'intent_vocab'))



def createModel(input_data, input_size, sequence_length, slots, slot_size, intent_size, layer_size=128,
                isTraining=True):
    cell_fw = tf.contrib.rnn.BasicLSTMCell(layer_size)
    cell_bw = tf.contrib.rnn.BasicLSTMCell(layer_size)

    if isTraining == True:
        cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, input_keep_prob=0.5,
                                                output_keep_prob=0.5)
        cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, input_keep_prob=0.5,
                                                output_keep_prob=0.5)
    # embedding layer， [word size, embed size] 724, 64
    if arg.embedding_path:
        embedding_weight = np.load(arg.embedding_path)
        embedding = tf.Variable(embedding_weight, name='embedding', dtype=tf.float32)
    else:
        embedding = tf.get_variable('embedding', [input_size, layer_size])
    # [bs, nstep, embed size]
    inputs = tf.nn.embedding_lookup(embedding, input_data)
    # state_outputs: [bs, nstep, embed size], final_state: [4, bs, embed size] include cell state * 2, hidden state * 2
    state_outputs, final_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, inputs,
                                                                 sequence_length=sequence_length, dtype=tf.float32)
    # [bs, embed size * 4]
    final_state = tf.concat([final_state[0][0], final_state[0][1], final_state[1][0], final_state[1][1]], 1)
    # [bs, nstep, embed size * 2]
    state_outputs = tf.concat([state_outputs[0], state_outputs[1]], 2)
    state_shape = state_outputs.get_shape()

    with tf.variable_scope('attention'):
        # [bs, nstep, embed size * 2]
        slot_inputs = state_outputs 
        if not remove_slot_attn:
            with tf.variable_scope('slot_attn'):
                # embed size * 2
                attn_size = state_shape[2].value
                origin_shape = tf.shape(state_outputs) 
                # [bs, 1, nstep, embed size * 2]
                hidden = tf.expand_dims(state_outputs, 1)
                # [bs, nstep, 1, embed size * 2]
                hidden_conv = tf.expand_dims(state_outputs, 2)
                # k: [filter_height, filter_width, in_channels, out_channels]
                k = tf.get_variable("AttnW", [1, 1, attn_size, attn_size])
                # [bs, nstep, 1, embed size * 2]
                hidden_features = tf.nn.conv2d(hidden_conv, k, [1, 1, 1, 1], "SAME")  
                # [bs, nstep, embed size * 2]
                hidden_features = tf.reshape(hidden_features, origin_shape)
                # [bs, 1, nstep, embed size * 2]
                hidden_features = tf.expand_dims(hidden_features, 1)
                v = tf.get_variable("AttnV", [attn_size])

                slot_inputs_shape = tf.shape(slot_inputs)
                # [bs * nstep, embed size * 2]
                slot_inputs = tf.reshape(slot_inputs, [-1, attn_size])
                # [bs * nstep, embed size * 2]
                y = core_rnn_cell._linear(slot_inputs, attn_size, True) 
                # [bs , nstep, embed size * 2]
                y = tf.reshape(y, slot_inputs_shape)
                # [bs , nstep, 1, embed size * 2]
                y = tf.expand_dims(y, 2)
                # [bs , nstep, nstep] = [bs, 1, nstep, hidden size] + [bs , nstep, 1, embed size * 2]
                s = tf.reduce_sum(v * tf.tanh(hidden_features + y), [3])
                a = tf.nn.softmax(s)
                # a shape = [bs, nstep, nstep, 1]
                a = tf.expand_dims(a, -1)
                # a shape = [bs, nstep, embed size * 2]
                slot_d = tf.reduce_sum(a * hidden, [2])
                slot_output = tf.reshape(slot_d,[-1,attn_size])
        else:
            attn_size = state_shape[2].value
            slot_d=state_outputs
            slot_inputs = tf.reshape(slot_inputs, [-1, attn_size])
            slot_output = slot_inputs

        intent_input = final_state
        with tf.variable_scope('intent_attn'):
            attn_size = state_shape[2].value 
            # [bs, nstep, 1, embed size * 2]
            hidden = tf.expand_dims(state_outputs, 2)
            k = tf.get_variable("AttnW", [1, 1, attn_size, attn_size])
            # [bs, nstep, 1, embed size * 2]
            hidden_features = tf.nn.conv2d(hidden, k, [1, 1, 1, 1], "SAME")
            v = tf.get_variable("AttnV", [attn_size])

            # [bs, embed size * 2]
            y = core_rnn_cell._linear(intent_input, attn_size, True)
            # [bs, 1, 1, embed size * 2]
            y = tf.reshape(y, [-1, 1, 1, attn_size])
            # [bs, nstep]
            s = tf.reduce_sum(v * tf.tanh(hidden_features + y), [2, 3])
            a = tf.nn.softmax(s)
            # [bs, nstep, 1]
            a = tf.expand_dims(a, -1)
            # [bs, nstep, 1, 1]
            a = tf.expand_dims(a, -1)
            # [bs, embed size * 2]
            d = tf.reduce_sum(a * hidden, [1, 2]) 
            intent_output = d
            #[bs, embedding * 2]
            intent_context_states = intent_output
            print(a)

        if arg.priority_order == 'intent_first':
            for n in range(arg.iteration_num):
                with tf.variable_scope('intent_subnet' + str(n - 1)):
                    # embedding*2
                    attn_size = state_shape[2].value
                    # [bs, nstep, 1, embed size * 2]
                    hidden = tf.expand_dims(state_outputs, 2)
                    # [bs,nstep, 1, embeddize*2]
                    reinforce_state = tf.expand_dims(slot_d, 2)
                    k1 = tf.get_variable("W1", [1, 1, attn_size, attn_size])
                    k2 = tf.get_variable('W2', [1, 1, attn_size, attn_size])
                    # [bs, nstep, 1, embed size * 2]
                    reinforce_features = tf.nn.conv2d(reinforce_state, k1, [1, 1, 1, 1], "SAME")
                    hidden_features = tf.nn.conv2d(hidden, k2, [1, 1, 1, 1], "SAME")
                    v1 = tf.get_variable("AttnV", [attn_size])
                    bias = tf.get_variable("Bias", [attn_size])
                    # [bs, nstep]
                    s = tf.reduce_sum(v1 * tf.tanh(hidden_features + reinforce_features + bias), [2, 3])
                    a = tf.nn.softmax(s)
                    # [bs, nstep, 1]
                    a = tf.expand_dims(a, -1)
                    # [bs, nstep, 1, 1]
                    a = tf.expand_dims(a, -1)
                    # [bs, embedding*2]
                    r_slot = tf.reduce_sum(a * reinforce_state, [1, 2])

                    r_intent = r_slot + intent_context_states

                    intent_output = tf.concat([r_intent, intent_input], 1)

                # with tf.variable_scope('slot_subnet' + str(n - 1)):
                #     # [bs, embed size * 2]
                #     intent_gate = core_rnn_cell._linear(intent_output, attn_size, True)
                #     # [bs, 1,embed size * 2]
                #     intent_gate = tf.reshape(intent_gate, [-1, 1, intent_gate.get_shape()[1].value])
                #     v1 = tf.get_variable("gateV", [attn_size])
                #     # [bs, nstep, embed size * 2]
                #     relation_factor = v1 * tf.tanh(slot_d + intent_gate)
                #     # [bs, nstep]
                #     relation_factor = tf.reduce_sum(relation_factor, [2])
                #     # [bs, nstep, 1]
                #     relation_factor = tf.expand_dims(relation_factor, -1)
                #     # [bs, nstep, embed size * 2]
                #     reinforce_state = slot_d * relation_factor
                #     # [bs * nstep, embed size * 2]
                #     reinforce_vector = tf.reshape(reinforce_state, [-1, attn_size])
                #     # [bs * nstep, embed size * 4]
                #     slot_output = tf.concat([reinforce_vector, slot_inputs], 1)
              

        else:
            for n in range(arg.iteration_num):
                # with tf.variable_scope('slot_subnet' + str(n - 1)):
                #     # [bs, embed size * 2]
                #     intent_gate = core_rnn_cell._linear(intent_output, attn_size, True)
                #     # [bs, 1,embed size * 2]
                #     intent_gate = tf.reshape(intent_gate, [-1, 1, intent_gate.get_shape()[1].value])  
                #     v1 = tf.get_variable("gateV", [attn_size])
                #     # [bs, nstep, embed size * 2]
                #     relation_factor = v1 * tf.tanh(slot_d + intent_gate)  
                #     # [bs, nstep]
                #     relation_factor = tf.reduce_sum(relation_factor, [2])
                #     # [bs, nstep, 1]
                #     relation_factor = tf.expand_dims(relation_factor, -1)
                #     reinforce_state = slot_d * relation_factor
                #     # [bs * nstep, embed size * 2]
                #     reinforce_vector = tf.reshape(reinforce_state, [-1, attn_size])
                #     # [bs * nstep, embed size * 4]
                #     slot_output = tf.concat([reinforce_vector,slot_inputs], 1)               

                with tf.variable_scope('intent_subnet' + str(n - 1)):
                    # embedding*2
                    attn_size = state_shape[2].value
                    # [bs, nstep, 1, embed size * 2]
                    hidden = tf.expand_dims(state_outputs, 2)
                    # [bs,nstep, 1, embedding_size*2]
                    reinforce_state = tf.expand_dims(slot_d, 2)

                    k1 = tf.get_variable("W1", [1, 1, attn_size, attn_size])
                    k2 = tf.get_variable('W2', [1, 1, attn_size, attn_size])
                    # [bs, nstep, 1, embed size * 2]
                    slot_features = tf.nn.conv2d(reinforce_state, k1, [1, 1, 1, 1], "SAME")
                    hidden_features = tf.nn.conv2d(hidden, k2, [1, 1, 1, 1], "SAME")
                    v1 = tf.get_variable("AttnV", [attn_size])
                    bias = tf.get_variable("Bias", [attn_size])
                    # [bs, nstep]
                    s = tf.reduce_sum(v1 * tf.tanh(hidden_features + slot_features + bias), [2, 3])
                    a = tf.nn.softmax(s)
                    # [bs, nstep, 1]
                    a = tf.expand_dims(a, -1)
                    # [bs, nstep, 1, 1]
                    a = tf.expand_dims(a, -1)
                    # [bs, embedding*2]
                    slot_reinforce_states = tf.reduce_sum(a * reinforce_state, [1, 2])

                    r_intent = slot_reinforce_states + intent_context_states

                    intent_output = tf.concat([r_intent, intent_input], 1)

    with tf.variable_scope('intent_proj'):
        # [bs, intent_size]
        intent = core_rnn_cell._linear(intent_output, intent_size, True)
    with tf.variable_scope('slot_proj'):
        # [bs * nsetp, intent_size]
        slot = core_rnn_cell._linear(slot_output, slot_size, True)
        if arg.use_crf:
            nstep = tf.shape(state_outputs)[1]
            slot = tf.reshape(slot, [-1, nstep, slot_size])
            # [bs,nstep,slot_size]
    outputs = [slot, intent]
    return outputs


# Create Training Model
input_data = tf.placeholder(tf.int32, [None, None], name='inputs')
sequence_length = tf.placeholder(tf.int32, [None], name="sequence_length")
global_step = tf.Variable(0, trainable=False, name='global_step')
slots = tf.placeholder(tf.int32, [None, None], name='slots')
slot_weights = tf.placeholder(tf.float32, [None, None], name='slot_weights')
intent = tf.placeholder(tf.int32, [None], name='intent')

with tf.variable_scope('model'):
    training_outputs = createModel(input_data, len(in_vocab['vocab']), sequence_length, slots, len(slot_vocab['vocab']),
                                   len(intent_vocab['vocab']), layer_size=arg.layer_size)

slots_shape = tf.shape(slots)
slots_reshape = tf.reshape(slots, [-1])

slot_outputs = training_outputs[0]
with tf.variable_scope('slot_loss'):
    if arg.use_crf:
        log_likelihood, trans_params = tf.contrib.crf.crf_log_likelihood(slot_outputs, slots, sequence_length)
        slot_loss = tf.reduce_mean(-log_likelihood) 
    else:
        crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=slots_reshape, logits=slot_outputs)
        crossent = tf.reshape(crossent, slots_shape)
        slot_loss = tf.reduce_sum(crossent * slot_weights, 1)
        total_size = tf.reduce_sum(slot_weights, 1)
        total_size += 1e-12
        slot_loss = slot_loss / total_size

intent_output = training_outputs[1]
with tf.variable_scope('intent_loss'):
    crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=intent, logits=intent_output)
    intent_loss = tf.reduce_sum(crossent) / tf.cast(arg.batch_size, tf.float32)
params = tf.trainable_variables()
# learning rate decay
learning_rate = tf.train.exponential_decay(arg.learning_rate, global_step, arg.decay_steps, arg.decay_rate,
                                           staircase=False)
if arg.learning_rate_decay:
    opt = tf.train.AdamOptimizer(learning_rate)
else:
    opt = tf.train.AdamOptimizer(arg.learning_rate)
intent_params = []
slot_params = []
for p in params:
    if not 'slot_' in p.name:
        intent_params.append(p)
    if 'slot_' in p.name or 'bidirectional_rnn' in p.name or 'embedding' in p.name:
        slot_params.append(p) 
gradients_slot = tf.gradients(slot_loss, slot_params)
gradients_intent = tf.gradients(intent_loss, intent_params)

clipped_gradients_slot, norm_slot = tf.clip_by_global_norm(gradients_slot, 5.0)
clipped_gradients_intent, norm_intent = tf.clip_by_global_norm(gradients_intent, 5.0)

gradient_norm_slot = norm_slot
gradient_norm_intent = norm_intent
update_slot = opt.apply_gradients(zip(clipped_gradients_slot, slot_params))
update_intent = opt.apply_gradients(zip(clipped_gradients_intent, intent_params), global_step=global_step)

training_outputs = [global_step, slot_loss, update_intent, update_slot, gradient_norm_intent, gradient_norm_slot]
inputs = [input_data, sequence_length, slots, slot_weights, intent]

# Create Inference Model
with tf.variable_scope('model', reuse=True):
    inference_outputs = createModel(input_data, len(in_vocab['vocab']), sequence_length, slots,
                                    len(slot_vocab['vocab']),
                                    len(intent_vocab['vocab']), layer_size=arg.layer_size, isTraining=False)
# slot output
if arg.use_crf:
    inference_slot_output, pred_scores = tf.contrib.crf.crf_decode(inference_outputs[0], trans_params, sequence_length)
else:
    inference_slot_output = tf.nn.softmax(inference_outputs[0], name='slot_output')
# intent output

inference_intent_output = tf.nn.softmax(inference_outputs[1], name='intent_output')

inference_outputs = [inference_intent_output, inference_slot_output]
inference_inputs = [input_data, sequence_length]

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

saver = tf.train.Saver()

# gpu setting
gpu_options = tf.GPUOptions(allow_growth=True)

# Start Training
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    sess.run(tf.global_variables_initializer()) 
    logging.info('Training Start')

    epochs = 0
    loss = 0.0
    data_processor = None
    line = 0
    num_loss = 0
    step = 0
    no_improve = 0

    # variables to store highest values among epochs, only use 'valid_err' for now
    valid_slot = 0
    test_slot = 0
    valid_intent = 0
    test_intent = 0
    valid_err = 0 
    test_err = 0
    best_epoch_num = 0
    while True:
        if data_processor == None:  
            data_processor = DataProcessor(os.path.join(full_train_path, arg.input_file),
                                           os.path.join(full_train_path, arg.slot_file),
                                           os.path.join(full_train_path, arg.intent_file), in_vocab, slot_vocab,
                                           intent_vocab)
        in_data, slot_data, slot_weight, length, intents, _, _, _ = data_processor.get_batch(arg.batch_size)
        feed_dict = {input_data.name: in_data, slots.name: slot_data, slot_weights.name: slot_weight,
                     sequence_length.name: length, intent.name: intents}
        ret = sess.run(training_outputs, feed_dict)
        loss += np.mean(ret[1])

        line += arg.batch_size
        step = ret[0] 
        num_loss += 1

        if data_processor.end == 1:
            arg.batch_size += arg.batch_size_add 
            line = 0
            #data_processor.close()
            data_processor = None
            epochs += 1
            logging.info('Step: ' + str(step))
            logging.info('Epochs: ' + str(epochs))
            logging.info('Loss: ' + str(loss / num_loss))
            num_loss = 0
            loss = 0.0

            save_path = os.path.join(arg.model_path, '_step_' + str(step) + '_epochs_' + str(epochs) + '.ckpt')
            saver.save(sess, save_path)


            def valid(in_path, slot_path, intent_path):
                data_processor_valid = DataProcessor(in_path, slot_path, intent_path, in_vocab, slot_vocab,
                                                     intent_vocab)

                pred_intents = []
                correct_intents = []
                slot_outputs = []
                correct_slots = []
                input_words = []

                # used to gate
                gate_seq = []
                while True:
                    in_data, slot_data, slot_weight, length, intents, in_seq, slot_seq, intent_seq = data_processor_valid.get_batch(
                        arg.batch_size)  
                    if len(in_data) <= 0:
                        break
                    feed_dict = {input_data.name: in_data, sequence_length.name: length}
                    ret = sess.run(inference_outputs, feed_dict)
                    for i in ret[0]: 
                        pred_intents.append(np.argmax(i))
                    for i in intents:
                        correct_intents.append(i)

                    pred_slots = ret[1].reshape((slot_data.shape[0], slot_data.shape[1], -1)) 
                    for p, t, i, l in zip(pred_slots, slot_data, in_data, length):
                        if arg.use_crf:
                            p = p.reshape([-1]) 
                        else:
                            p = np.argmax(p, 1)
                        tmp_pred = []
                        tmp_correct = []
                        tmp_input = []
                        for j in range(l):
                            tmp_pred.append(slot_vocab['rev'][p[j]])
                            tmp_correct.append(slot_vocab['rev'][t[j]])
                            tmp_input.append(in_vocab['rev'][i[j]])

                        slot_outputs.append(tmp_pred)
                        correct_slots.append(tmp_correct)
                        input_words.append(tmp_input)

                    if data_processor_valid.end == 1:
                        break

                pred_intents = np.array(pred_intents)
                correct_intents = np.array(correct_intents)
                accuracy = (pred_intents == correct_intents)
                semantic_acc = accuracy
                accuracy = accuracy.astype(float)
                accuracy = np.mean(accuracy) * 100.0

                index = 0
                for t, p in zip(correct_slots, slot_outputs):
                    # Process Semantic Error
                    if len(t) != len(p):
                        raise ValueError('Error!!')

                    for j in range(len(t)):
                        if p[j] != t[j]:
                            semantic_acc[index] = False
                            break
                    index += 1
                semantic_acc = semantic_acc.astype(float)
                semantic_acc = np.mean(semantic_acc) * 100.0

                f1, precision, recall = computeF1Score(correct_slots, slot_outputs)
                logging.info('slot f1: ' + str(f1))
                logging.info('intent accuracy: ' + str(accuracy))
                logging.info('semantic Acc(intent, slots are all correct): ' + str(semantic_acc))

                #data_processor_valid.close()
                return f1, accuracy, semantic_acc, pred_intents, correct_intents, slot_outputs, correct_slots, input_words, gate_seq


            logging.info('Valid:')
            epoch_valid_slot, epoch_valid_intent, epoch_valid_err, valid_pred_intent, valid_correct_intent, valid_pred_slot, valid_correct_slot, valid_words, valid_gate = valid(
                os.path.join(full_valid_path, arg.input_file), os.path.join(full_valid_path, arg.slot_file),
                os.path.join(full_valid_path, arg.intent_file))

            logging.info('Test:')
            epoch_test_slot, epoch_test_intent, epoch_test_err, test_pred_intent, test_correct_intent, test_pred_slot, test_correct_slot, test_words, test_gate = valid(
                os.path.join(full_test_path, arg.input_file), os.path.join(full_test_path, arg.slot_file),
                os.path.join(full_test_path, arg.intent_file))

            if epoch_test_err <= test_err:
                no_improve += 1
            else:
                best_epoch_num = epochs
                test_err = epoch_test_err

                # logging.info('new best epoch number: Epoch Number: {}'.format(best_epoch_num))
                # logging.info('new best score: Semantic Acc: {}'.format(epoch_test_err))
                no_improve = 0

            if test_err > 0:
                logging.info('best epoch_num :  {}'.format(best_epoch_num))
                logging.info('best score : {}'.format(test_err))

            if epochs == arg.max_epochs:
                break

            if arg.early_stop == True:
                if no_improve > arg.patience:
                    break

batch_size = 16
batch_size_add = 4
cell = lstm
dataset = snips
decay_rate = 0.9
decay_steps = 1120
early_stop = True
embedding_path = 
input_file = seq.in
intent_file = label
iteration_num = 1
layer_size = 64
learning_rate = 0.001
learning_rate_decay = 1
max_epochs = 100
model_path = ./model
model_type = full
patience = 15
priority_order = slot_first
slot_file = seq.out
test_data_path = test
train_data_path = train
use_crf = True
use_embedding = 1
valid_data_path = valid
vocab_path = ./vocab

use snips dataset
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorf

2020-07-26 04:34:02,042 : INFO : Training Start
2020-07-26 04:34:50,857 : INFO : Step: 818
2020-07-26 04:34:50,858 : INFO : Epochs: 1
2020-07-26 04:34:50,860 : INFO : Loss: 13.440481363706892
2020-07-26 04:34:51,431 : INFO : Valid:
2020-07-26 04:34:54,612 : INFO : slot f1: 37.565521258008154
2020-07-26 04:34:54,613 : INFO : intent accuracy: 97.57142857142857
2020-07-26 04:34:54,614 : INFO : semantic Acc(intent, slots are all correct): 17.42857142857143
2020-07-26 04:34:54,622 : INFO : Test:
2020-07-26 04:34:57,685 : INFO : slot f1: 34.7877358490566
2020-07-26 04:34:57,686 : INFO : intent accuracy: 96.28571428571429
2020-07-26 04:34:57,687 : INFO : semantic Acc(intent, slots are all correct): 15.285714285714286
2020-07-26 04:34:57,689 : INFO : best epoch_num :  1
2020-07-26 04:34:57,690 : INFO : best score : 15.285714285714286
2020-07-26 04:35:40,893 : INFO : Step: 1473
2020-07-26 04:35:40,894 : INFO : Epochs: 2
2020-07-26 04:35:40,895 : INFO : Loss: 6.91541067807729
2020-07-26 04:35:41

Instructions for updating:
Use standard file APIs to delete files with this prefix.


2020-07-26 04:38:12,670 : WARNING : From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/training/saver.py:963: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
2020-07-26 04:38:12,986 : INFO : Valid:
2020-07-26 04:38:13,442 : INFO : slot f1: 69.08678389109473
2020-07-26 04:38:13,443 : INFO : intent accuracy: 99.28571428571429
2020-07-26 04:38:13,443 : INFO : semantic Acc(intent, slots are all correct): 48.857142857142854
2020-07-26 04:38:13,446 : INFO : Test:
2020-07-26 04:38:13,988 : INFO : slot f1: 67.50142287990893
2020-07-26 04:38:13,989 : INFO : intent accuracy: 97.71428571428571
2020-07-26 04:38:13,991 : INFO : semantic Acc(intent, slots are all correct): 47.14285714285714
2020-07-26 04:38:13,992 : INFO : best epoch_num :  6
2020-07-26 04:38:13,993 : INFO : best score : 47.14285714285714
2020-07-26 04:38

In [ ]:
import os
import argparse
import logging
import sys
import tensorflow as tf
import numpy as np
from tensorflow.contrib.rnn.python.ops import core_rnn_cell
from tensorflow.python.ops import rnn_cell_impl

#from utils import createVocabulary, loadVocabulary, computeF1Score, DataProcessor

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# tf.set_random_seed(20181226) 
# np.random.seed(20181226)
# todo: 1. word pre-train embedding, gru, crf, lr decay

parser = argparse.ArgumentParser(allow_abbrev=False)

# Network
parser.add_argument("--num_units", type=int, default=64, help="Network size.", dest='layer_size',required=False)
parser.add_argument("--model_type", type=str, default='full', help="""full(default) | intent_only
                                                                    full: full attention model
                                                                    intent_only: intent attention model""",required=False)
parser.add_argument("--priority_order", type=str, default='slot_first', help="""Type 'slot_first' or 'intent_first'
                                                                              to decide whose influence ought to calculate first use.""",required=False)
parser.add_argument("--use_crf", type=bool, default=True, help="""use crf for seq labeling""",required=False)
parser.add_argument("--use_embedding", type=str, default='1', help="""use pre-trained embedding""",required=False)
parser.add_argument("--cell", type=str, default='lstm', help="""rnn cell""",required=False)  
parser.add_argument("--iteration_num", type=int, default=1, help="""the number of iteration times""",required=False)

# Training Environment
parser.add_argument("--batch_size", type=int, default=16, help="Batch size.",required=False)
parser.add_argument("--batch_size_add", type=int, default=4, help="Batch size add.",required=False)
parser.add_argument("--max_epochs", type=int, default=100, help="Max epochs to train.",required=False)
parser.add_argument("--no_early_stop", action='store_false', dest='early_stop',
                    help="Disable early stop, which is based on sentence level accuracy.",required=False)  
parser.add_argument("--patience", type=int, default=15, help="Patience to wait before stop.",required=False)
# learn rate param
parser.add_argument("--learning_rate_decay", type=str, default='1', help="learning_rate_decay",required=False)
parser.add_argument("--learning_rate", type=float, default=0.001, help="The initial learning rate.",required=False)
parser.add_argument("--decay_steps", type=int, default=280 * 4, help="decay_steps.",required=False)
parser.add_argument("--decay_rate", type=float, default=0.9, help="decay_rate.",required=False)

# Model and Vocab
parser.add_argument("--dataset", type=str, default='snips', help="""Type 'atis' or 'snips' to use dataset provided by us or enter what ever you named your own dataset.
                Note, if you don't want to use this part, enter --dataset=''. It can not be None""",required=False)
parser.add_argument("--model_path", type=str, default='./model', help="Path to save model.",required=False)
parser.add_argument("--vocab_path", type=str, default='./vocab', help="Path to vocabulary files.",required=False)

# Data
parser.add_argument("--train_data_path", type=str, default='train', help="Path to training data files.",required=False)
parser.add_argument("--test_data_path", type=str, default='test', help="Path to testing data files.",required=False)
parser.add_argument("--valid_data_path", type=str, default='valid', help="Path to validation data files.",required=False)
parser.add_argument("--input_file", type=str, default='seq.in', help="Input file name.",required=False)
parser.add_argument("--slot_file", type=str, default='seq.out', help="Slot file name.",required=False)
parser.add_argument("--intent_file", type=str, default='label', help="Intent file name.",required=False)
parser.add_argument("--embedding_path", type=str, default='', help="embedding array's path.",required=False)

arg = parser.parse_args(''.split())

if arg.dataset=='atis':
    arg.model_type='intent_only'
else:
    arg.model_type='full'

# Print arguments
for k, v in sorted(vars(arg).items()):
    print(k, '=', v)
print()
# use full attention or intent only
if arg.model_type == 'full':
    remove_slot_attn = False
elif arg.model_type == 'intent_only':
    remove_slot_attn = True
else:
    print('unknown model type!')
    exit(1)

# full path to data will be: ./data + dataset + train/test/valid
if arg.dataset == None:
    print('name of dataset can not be None')
    exit(1)
elif arg.dataset == 'snips':
    print('use snips dataset')
elif arg.dataset == 'atis':
    print('use atis dataset')
else:
    print('use own dataset: ', arg.dataset)
full_train_path = os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/data', arg.dataset, arg.train_data_path)
full_test_path = os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/data', arg.dataset, arg.test_data_path)
full_valid_path = os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/data', arg.dataset, arg.valid_data_path)

createVocabulary(os.path.join(full_train_path, arg.input_file), os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'in_vocab'))
createVocabulary(os.path.join(full_train_path, arg.slot_file), os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'slot_vocab'))
createVocabulary(os.path.join(full_train_path, arg.intent_file), os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'intent_vocab'))
# return map: {'vocab': vocab, 'rev': rev}, vocab: map, rev: array
in_vocab = loadVocabulary(os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'in_vocab'))
slot_vocab = loadVocabulary(os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'slot_vocab'))
intent_vocab = loadVocabulary(os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'intent_vocab'))


def createModel(input_data, input_size, sequence_length, slots, slot_size, intent_size, layer_size=128,
                isTraining=True):
    cell_fw = tf.contrib.rnn.BasicLSTMCell(layer_size)
    cell_bw = tf.contrib.rnn.BasicLSTMCell(layer_size)

    if isTraining == True:
        cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, input_keep_prob=0.5,
                                                output_keep_prob=0.5)
        cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, input_keep_prob=0.5,
                                                output_keep_prob=0.5)
    # embedding layer， [word size, embed size] 724, 64
    if arg.embedding_path:  
        embedding_weight = np.load(arg.embedding_path)
        embedding = tf.Variable(embedding_weight, name='embedding', dtype=tf.float32)
    else:
        embedding = tf.get_variable('embedding', [input_size, layer_size])
    # [bs, nstep, embed size]
    inputs = tf.nn.embedding_lookup(embedding, input_data)
    # state_outputs: [bs, nstep, embed size], final_state: [4, bs, embed size] include cell state * 2, hidden state * 2
    state_outputs, final_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, inputs,
                                                                 sequence_length=sequence_length, dtype=tf.float32)
    # [bs, embed size * 4]
    final_state = tf.concat([final_state[0][0], final_state[0][1], final_state[1][0], final_state[1][1]], 1)
    # [bs, nstep, embed size * 2]
    state_outputs = tf.concat([state_outputs[0], state_outputs[1]], 2)
    state_shape = state_outputs.get_shape()

    with tf.variable_scope('attention'):
        # [bs, nstep, embed size * 2]
        slot_inputs = state_outputs 
        if not remove_slot_attn:
            with tf.variable_scope('slot_attn'):
                # embed size * 2
                attn_size = state_shape[2].value
                origin_shape = tf.shape(state_outputs) 
                # [bs, 1, nstep, embed size * 2]
                hidden = tf.expand_dims(state_outputs, 1)
                # [bs, nstep, 1, embed size * 2]
                hidden_conv = tf.expand_dims(state_outputs, 2)
                # k: [filter_height, filter_width, in_channels, out_channels]
                k = tf.get_variable("AttnW", [1, 1, attn_size, attn_size])
                # [bs, nstep, 1, embed size * 2]
                hidden_features = tf.nn.conv2d(hidden_conv, k, [1, 1, 1, 1], "SAME") 
                # [bs, nstep, embed size * 2]
                hidden_features = tf.reshape(hidden_features, origin_shape)
                # [bs, 1, nstep, embed size * 2]
                hidden_features = tf.expand_dims(hidden_features, 1)
                v = tf.get_variable("AttnV", [attn_size])

                slot_inputs_shape = tf.shape(slot_inputs)
                # [bs * nstep, embed size * 2]
                slot_inputs = tf.reshape(slot_inputs, [-1, attn_size])
                # [bs * nstep, embed size * 2]
                y = core_rnn_cell._linear(slot_inputs, attn_size, True)
                # [bs , nstep, embed size * 2]
                y = tf.reshape(y, slot_inputs_shape)
                # [bs , nstep, 1, embed size * 2]
                y = tf.expand_dims(y, 2)
                # [bs , nstep, nstep] = [bs, 1, nstep, hidden size] + [bs , nstep, 1, embed size * 2]
                s = tf.reduce_sum(v * tf.tanh(hidden_features + y), [3])
                a = tf.nn.softmax(s)
                # a shape = [bs, nstep, nstep, 1]
                a = tf.expand_dims(a, -1)
                # a shape = [bs, nstep, embed size * 2]
                slot_d = tf.reduce_sum(a * hidden, [2])
        else:
            attn_size = state_shape[2].value
            slot_d=state_outputs
            slot_inputs = tf.reshape(slot_inputs, [-1, attn_size])

        intent_input = final_state
        with tf.variable_scope('intent_attn'):
            attn_size = state_shape[2].value  
            # [bs, nstep, 1, embed size * 2]
            hidden = tf.expand_dims(state_outputs, 2)
            k = tf.get_variable("AttnW", [1, 1, attn_size, attn_size])
            # [bs, nstep, 1, embed size * 2]
            hidden_features = tf.nn.conv2d(hidden, k, [1, 1, 1, 1], "SAME")
            v = tf.get_variable("AttnV", [attn_size])

            # [bs, embed size * 2]
            y = core_rnn_cell._linear(intent_input, attn_size, True)
            # [bs, 1, 1, embed size * 2]
            y = tf.reshape(y, [-1, 1, 1, attn_size])
            # [bs, nstep]
            s = tf.reduce_sum(v * tf.tanh(hidden_features + y), [2, 3])  
            a = tf.nn.softmax(s)
            # [bs, nstep, 1]
            a = tf.expand_dims(a, -1)
            # [bs, nstep, 1, 1]
            a = tf.expand_dims(a, -1)
            # [bs, embed size * 2]
            d = tf.reduce_sum(a * hidden, [1, 2])
            intent_output = d
            #[bs, embedding * 2]
            intent_context_states = intent_output
            print(a)

        if arg.priority_order == 'intent_first':
            for n in range(arg.iteration_num):
                # with tf.variable_scope('intent_subnet' + str(n - 1)):
                #     # embedding*2
                #     attn_size = state_shape[2].value
                #     # [bs, nstep, 1, embed size * 2]
                #     hidden = tf.expand_dims(state_outputs, 2)
                #     # [bs,nstep, 1, embeddize*2]
                #     reinforce_state = tf.expand_dims(slot_d, 2)
                #     k1 = tf.get_variable("W1", [1, 1, attn_size, attn_size])
                #     k2 = tf.get_variable('W2', [1, 1, attn_size, attn_size])
                #     # [bs, nstep, 1, embed size * 2]
                #     reinforce_features = tf.nn.conv2d(reinforce_state, k1, [1, 1, 1, 1], "SAME")
                #     hidden_features = tf.nn.conv2d(hidden, k2, [1, 1, 1, 1], "SAME")
                #     v1 = tf.get_variable("AttnV", [attn_size])
                #     bias = tf.get_variable("Bias", [attn_size])
                #     # [bs, nstep]
                #     s = tf.reduce_sum(v1 * tf.tanh(hidden_features + reinforce_features + bias), [2, 3])
                #     a = tf.nn.softmax(s)
                #     # [bs, nstep, 1]
                #     a = tf.expand_dims(a, -1)
                #     # [bs, nstep, 1, 1]
                #     a = tf.expand_dims(a, -1)
                #     # [bs, embedding*2]
                #     r_slot = tf.reduce_sum(a * reinforce_state, [1, 2])
                #
                #     r_intent = r_slot + intent_context_states
                #
                #     intent_output = tf.concat([r_intent, intent_input], 1)

                with tf.variable_scope('slot_subnet' + str(n - 1)):
                    # [bs, embed size * 2]
                    intent_gate = core_rnn_cell._linear(intent_output, attn_size, True)
                    # [bs, 1,embed size * 2]
                    intent_gate = tf.reshape(intent_gate, [-1, 1, intent_gate.get_shape()[1].value]) 
                    v1 = tf.get_variable("gateV", [attn_size])
                    # [bs, nstep, embed size * 2]
                    relation_factor = v1 * tf.tanh(slot_d + intent_gate)
                    # [bs, nstep]
                    relation_factor = tf.reduce_sum(relation_factor, [2])
                    # [bs, nstep, 1]
                    relation_factor = tf.expand_dims(relation_factor, -1)
                    # [bs, nstep, embed size * 2]
                    reinforce_state = slot_d * relation_factor
                    # [bs * nstep, embed size * 2]
                    reinforce_vector = tf.reshape(reinforce_state, [-1, attn_size])
                    # [bs * nstep, embed size * 4]
                    slot_output = tf.concat([reinforce_vector, slot_inputs], 1)


        else:
            for n in range(arg.iteration_num):
                with tf.variable_scope('slot_subnet' + str(n - 1)):
                    # [bs, embed size * 2]
                    intent_gate = core_rnn_cell._linear(intent_output, attn_size, True)
                    # [bs, 1,embed size * 2]
                    intent_gate = tf.reshape(intent_gate, [-1, 1, intent_gate.get_shape()[1].value]) 
                    v1 = tf.get_variable("gateV", [attn_size])
                    # [bs, nstep, embed size * 2]
                    relation_factor = v1 * tf.tanh(slot_d + intent_gate) 
                    # [bs, nstep]
                    relation_factor = tf.reduce_sum(relation_factor, [2])
                    # [bs, nstep, 1]
                    relation_factor = tf.expand_dims(relation_factor, -1)
                    reinforce_state = slot_d * relation_factor
                    # [bs * nstep, embed size * 2]
                    reinforce_vector = tf.reshape(reinforce_state, [-1, attn_size])
                    # [bs * nstep, embed size * 4]
                    slot_output = tf.concat([reinforce_vector,slot_inputs], 1)                    

                # with tf.variable_scope('intent_subnet' + str(n - 1)):
                #     # embedding*2
                #     attn_size = state_shape[2].value
                #     # [bs, nstep, 1, embed size * 2]
                #     hidden = tf.expand_dims(state_outputs, 2)
                #     # [bs,nstep, 1, embedding_size*2]
                #     reinforce_output = tf.expand_dims(reinforce_state, 2)
                #     k1 = tf.get_variable("W1", [1, 1, attn_size, attn_size])
                #     k2 = tf.get_variable('W2', [1, 1, attn_size, attn_size])
                #     # [bs, nstep, 1, embed size * 2]
                #     slot_features = tf.nn.conv2d(reinforce_output, k1, [1, 1, 1, 1], "SAME")  
                #     hidden_features = tf.nn.conv2d(hidden, k2, [1, 1, 1, 1], "SAME")
                #     v1 = tf.get_variable("AttnV", [attn_size])
                #     bias = tf.get_variable("Bias", [attn_size])
                #     # [bs, nstep]
                #     s = tf.reduce_sum(v1 * tf.tanh(hidden_features + slot_features + bias), [2, 3])
                #     a = tf.nn.softmax(s)
                #     # [bs, nstep, 1]
                #     a = tf.expand_dims(a, -1)
                #     # [bs, nstep, 1, 1]
                #     a = tf.expand_dims(a, -1)
                #     # [bs, embedding*2]
                #     slot_reinforce_states = tf.reduce_sum(a * reinforce_output, [1, 2])
                #
                #     r_intent = slot_reinforce_states + intent_context_states
                #
                #     intent_output = tf.concat([r_intent, intent_input], 1)

    with tf.variable_scope('intent_proj'):
        # [bs, intent_size]
        intent = core_rnn_cell._linear(intent_output, intent_size, True)
    with tf.variable_scope('slot_proj'):
        # [bs * nsetp, intent_size]
        slot = core_rnn_cell._linear(slot_output, slot_size, True)
        if arg.use_crf:
            nstep = tf.shape(state_outputs)[1]
            slot = tf.reshape(slot, [-1, nstep, slot_size])
            # [bs,nstep,slot_size]
    outputs = [slot, intent]
    return outputs


# Create Training Model
input_data = tf.placeholder(tf.int32, [None, None], name='inputs') 
sequence_length = tf.placeholder(tf.int32, [None], name="sequence_length")
global_step = tf.Variable(0, trainable=False, name='global_step')
slots = tf.placeholder(tf.int32, [None, None], name='slots')
slot_weights = tf.placeholder(tf.float32, [None, None], name='slot_weights')
intent = tf.placeholder(tf.int32, [None], name='intent')

with tf.variable_scope('model'):
    training_outputs = createModel(input_data, len(in_vocab['vocab']), sequence_length, slots, len(slot_vocab['vocab']),
                                   len(intent_vocab['vocab']), layer_size=arg.layer_size)

slots_shape = tf.shape(slots)
slots_reshape = tf.reshape(slots, [-1])

slot_outputs = training_outputs[0]
with tf.variable_scope('slot_loss'):
    if arg.use_crf:
        log_likelihood, trans_params = tf.contrib.crf.crf_log_likelihood(slot_outputs, slots, sequence_length)
        slot_loss = tf.reduce_mean(-log_likelihood)
    else:
        crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=slots_reshape, logits=slot_outputs)
        crossent = tf.reshape(crossent, slots_shape)
        slot_loss = tf.reduce_sum(crossent * slot_weights, 1)
        total_size = tf.reduce_sum(slot_weights, 1)
        total_size += 1e-12
        slot_loss = slot_loss / total_size

intent_output = training_outputs[1]
with tf.variable_scope('intent_loss'):
    crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=intent, logits=intent_output)
    intent_loss = tf.reduce_sum(crossent) / tf.cast(arg.batch_size, tf.float32)

params = tf.trainable_variables()
# learning rate decay
learning_rate = tf.train.exponential_decay(arg.learning_rate, global_step, arg.decay_steps, arg.decay_rate,
                                           staircase=False)
if arg.learning_rate_decay:
    opt = tf.train.AdamOptimizer(learning_rate)
else:
    opt = tf.train.AdamOptimizer(arg.learning_rate)
intent_params = []
slot_params = []
for p in params:
    if not 'slot_' in p.name:
        intent_params.append(p)
    if 'slot_' in p.name or 'bidirectional_rnn' in p.name or 'embedding' in p.name:
        slot_params.append(p)  

gradients_slot = tf.gradients(slot_loss, slot_params)
gradients_intent = tf.gradients(intent_loss, intent_params)
clipped_gradients_slot, norm_slot = tf.clip_by_global_norm(gradients_slot, 5.0)
clipped_gradients_intent, norm_intent = tf.clip_by_global_norm(gradients_intent, 5.0)
gradient_norm_slot = norm_slot
gradient_norm_intent = norm_intent
update_slot = opt.apply_gradients(zip(clipped_gradients_slot, slot_params))
update_intent = opt.apply_gradients(zip(clipped_gradients_intent, intent_params), global_step=global_step)
training_outputs = [global_step, slot_loss, update_intent, update_slot, gradient_norm_intent, gradient_norm_slot]
inputs = [input_data, sequence_length, slots, slot_weights, intent]

# Create Inference Model
with tf.variable_scope('model', reuse=True):
    inference_outputs = createModel(input_data, len(in_vocab['vocab']), sequence_length, slots,
                                    len(slot_vocab['vocab']),
                                    len(intent_vocab['vocab']), layer_size=arg.layer_size, isTraining=False)
# slot output
if arg.use_crf:
    inference_slot_output, pred_scores = tf.contrib.crf.crf_decode(inference_outputs[0], trans_params, sequence_length)
else:
    inference_slot_output = tf.nn.softmax(inference_outputs[0], name='slot_output')
# intent output

inference_intent_output = tf.nn.softmax(inference_outputs[1], name='intent_output')

inference_outputs = [inference_intent_output, inference_slot_output]
inference_inputs = [input_data, sequence_length]

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

saver = tf.train.Saver()
# gpu setting
gpu_options = tf.GPUOptions(allow_growth=True)

# Start Training
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    sess.run(tf.global_variables_initializer()) 
    logging.info('Training Start') 

    epochs = 0
    loss = 0.0
    data_processor = None
    line = 0
    num_loss = 0
    step = 0
    no_improve = 0

    # variables to store highest values among epochs, only use 'valid_err' for now
    valid_slot = 0
    test_slot = 0
    valid_intent = 0
    test_intent = 0
    valid_err = 0  
    test_err = 0
    best_epoch_num = 0
    while True:
        if data_processor == None:
            data_processor = DataProcessor(os.path.join(full_train_path, arg.input_file),
                                           os.path.join(full_train_path, arg.slot_file),
                                           os.path.join(full_train_path, arg.intent_file), in_vocab, slot_vocab,
                                           intent_vocab)
        in_data, slot_data, slot_weight, length, intents, _, _, _ = data_processor.get_batch(arg.batch_size)
        feed_dict = {input_data.name: in_data, slots.name: slot_data, slot_weights.name: slot_weight,
                     sequence_length.name: length, intent.name: intents}
        ret = sess.run(training_outputs, feed_dict)
        loss += np.mean(ret[1])

        line += arg.batch_size
        step = ret[0]
        num_loss += 1

        if data_processor.end == 1:
            arg.batch_size += arg.batch_size_add 
            line = 0
            #data_processor.close() 
            data_processor = None
            epochs += 1 
            logging.info('Step: ' + str(step))
            logging.info('Epochs: ' + str(epochs))
            logging.info('Loss: ' + str(loss / num_loss))
            num_loss = 0
            loss = 0.0

            save_path = os.path.join(arg.model_path, '_step_' + str(step) + '_epochs_' + str(epochs) + '.ckpt')
            saver.save(sess, save_path)


            def valid(in_path, slot_path, intent_path):
                data_processor_valid = DataProcessor(in_path, slot_path, intent_path, in_vocab, slot_vocab,
                                                     intent_vocab)

                pred_intents = []
                correct_intents = []
                slot_outputs = []
                correct_slots = []
                input_words = []

                # used to gate
                gate_seq = []
                while True:
                    in_data, slot_data, slot_weight, length, intents, in_seq, slot_seq, intent_seq = data_processor_valid.get_batch(
                        arg.batch_size)
                    if len(in_data) <= 0:
                        break
                    feed_dict = {input_data.name: in_data, sequence_length.name: length}
                    ret = sess.run(inference_outputs, feed_dict)
                    for i in ret[0]:  
                        pred_intents.append(np.argmax(i))
                    for i in intents:
                        correct_intents.append(i)

                    pred_slots = ret[1].reshape((slot_data.shape[0], slot_data.shape[1], -1))
                    for p, t, i, l in zip(pred_slots, slot_data, in_data, length):
                        if arg.use_crf:
                            p = p.reshape([-1])
                        else:
                            p = np.argmax(p, 1)
                        tmp_pred = []
                        tmp_correct = []
                        tmp_input = []
                        for j in range(l):
                            tmp_pred.append(slot_vocab['rev'][p[j]])
                            tmp_correct.append(slot_vocab['rev'][t[j]])
                            tmp_input.append(in_vocab['rev'][i[j]])

                        slot_outputs.append(tmp_pred)  
                        correct_slots.append(tmp_correct)
                        input_words.append(tmp_input)

                    if data_processor_valid.end == 1:
                        break

                pred_intents = np.array(pred_intents)
                correct_intents = np.array(correct_intents)
                accuracy = (pred_intents == correct_intents)
                semantic_acc = accuracy
                accuracy = accuracy.astype(float)
                accuracy = np.mean(accuracy) * 100.0  

                index = 0
                for t, p in zip(correct_slots, slot_outputs):
                    # Process Semantic Error
                    if len(t) != len(p):
                        raise ValueError('Error!!')

                    for j in range(len(t)):
                        if p[j] != t[j]:
                            semantic_acc[index] = False
                            break
                    index += 1
                semantic_acc = semantic_acc.astype(float)
                semantic_acc = np.mean(semantic_acc) * 100.0

                f1, precision, recall = computeF1Score(correct_slots, slot_outputs)
                logging.info('slot f1: ' + str(f1))
                logging.info('intent accuracy: ' + str(accuracy))
                logging.info('semantic Acc(intent, slots are all correct): ' + str(semantic_acc))

                #data_processor_valid.close()
                return f1, accuracy, semantic_acc, pred_intents, correct_intents, slot_outputs, correct_slots, input_words, gate_seq


            logging.info('Valid:')
            epoch_valid_slot, epoch_valid_intent, epoch_valid_err, valid_pred_intent, valid_correct_intent, valid_pred_slot, valid_correct_slot, valid_words, valid_gate = valid(
                os.path.join(full_valid_path, arg.input_file), os.path.join(full_valid_path, arg.slot_file),
                os.path.join(full_valid_path, arg.intent_file))

            logging.info('Test:')
            epoch_test_slot, epoch_test_intent, epoch_test_err, test_pred_intent, test_correct_intent, test_pred_slot, test_correct_slot, test_words, test_gate = valid(
                os.path.join(full_test_path, arg.input_file), os.path.join(full_test_path, arg.slot_file),
                os.path.join(full_test_path, arg.intent_file))

            if epoch_test_err <= test_err:
                no_improve += 1
            else:
                best_epoch_num = epochs
                test_err = epoch_test_err

                # logging.info('new best epoch number: Epoch Number: {}'.format(best_epoch_num))
                # logging.info('new best score: Semantic Acc: {}'.format(epoch_test_err))
                no_improve = 0

            if test_err > 0:
                logging.info('best epoch_num :  {}'.format(best_epoch_num))
                logging.info('best score : {}'.format(test_err))

            if epochs == arg.max_epochs:
                break

            if arg.early_stop == True:
                if no_improve > arg.patience:
                    break

batch_size = 16
batch_size_add = 4
cell = lstm
dataset = snips
decay_rate = 0.9
decay_steps = 1120
early_stop = True
embedding_path = 
input_file = seq.in
intent_file = label
iteration_num = 1
layer_size = 64
learning_rate = 0.001
learning_rate_decay = 1
max_epochs = 100
model_path = ./model
model_type = full
patience = 15
priority_order = slot_first
slot_file = seq.out
test_data_path = test
train_data_path = train
use_crf = True
use_embedding = 1
valid_data_path = valid
vocab_path = ./vocab

use snips dataset
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorf

2020-07-26 05:40:45,459 : INFO : Training Start
2020-07-26 05:41:37,679 : INFO : Step: 818
2020-07-26 05:41:37,681 : INFO : Epochs: 1
2020-07-26 05:41:37,684 : INFO : Loss: 11.490834472523343
2020-07-26 05:41:38,173 : INFO : Valid:
2020-07-26 05:41:39,106 : INFO : slot f1: 66.00734670810962
2020-07-26 05:41:39,108 : INFO : intent accuracy: 98.0
2020-07-26 05:41:39,109 : INFO : semantic Acc(intent, slots are all correct): 40.14285714285714
2020-07-26 05:41:39,110 : INFO : Test:
2020-07-26 05:41:40,029 : INFO : slot f1: 65.3141730064807
2020-07-26 05:41:40,030 : INFO : intent accuracy: 95.57142857142857
2020-07-26 05:41:40,034 : INFO : semantic Acc(intent, slots are all correct): 40.14285714285714
2020-07-26 05:41:40,036 : INFO : best epoch_num :  1
2020-07-26 05:41:40,037 : INFO : best score : 40.14285714285714
2020-07-26 05:42:25,287 : INFO : Step: 1473
2020-07-26 05:42:25,289 : INFO : Epochs: 2
2020-07-26 05:42:25,290 : INFO : Loss: 3.517556041797609
2020-07-26 05:42:25,613 : INFO : V

Instructions for updating:
Use standard file APIs to delete files with this prefix.


2020-07-26 05:45:05,934 : WARNING : From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/training/saver.py:963: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
2020-07-26 05:45:06,250 : INFO : Valid:
2020-07-26 05:45:06,739 : INFO : slot f1: 90.57127010537991
2020-07-26 05:45:06,739 : INFO : intent accuracy: 99.14285714285714
2020-07-26 05:45:06,741 : INFO : semantic Acc(intent, slots are all correct): 77.14285714285715
2020-07-26 05:45:06,743 : INFO : Test:
2020-07-26 05:45:07,332 : INFO : slot f1: 89.06725712704126
2020-07-26 05:45:07,336 : INFO : intent accuracy: 97.28571428571429
2020-07-26 05:45:07,337 : INFO : semantic Acc(intent, slots are all correct): 73.85714285714286
2020-07-26 05:45:07,340 : INFO : best epoch_num :  6
2020-07-26 05:45:07,342 : INFO : best score : 73.85714285714286
2020-07-26 05:45:

In [ ]:
import os
import argparse
import logging
import sys
import tensorflow as tf
import numpy as np
from tensorflow.contrib.rnn.python.ops import core_rnn_cell
from tensorflow.python.ops import rnn_cell_impl

#from utils import createVocabulary, loadVocabulary, computeF1Score, DataProcessor

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

parser = argparse.ArgumentParser(allow_abbrev=False)
parser.add_argument("--num_units", type=int, default=64, help="Network size.", dest='layer_size',required=False)
parser.add_argument("--model_type", type=str, default='full', help="""full(default) | intent_only
                                                                    full: full attention model
                                                                    intent_only: intent attention model""",required=False)
parser.add_argument("--priority_order", type=str, default='slot_first', help="""Type 'slot_first' or 'intent_first'
                                                                              to decide whose influence ought to calculate first use.""",required=False)
parser.add_argument("--use_crf", type=bool, default=False, help="""use crf for seq labeling""",required=False)
parser.add_argument("--use_embedding", type=str, default='1', help="""use pre-trained embedding""",required=False)
parser.add_argument("--cell", type=str, default='lstm', help="""rnn cell""",required=False)
parser.add_argument("--iteration_num", type=int, default=1, help="""the number of iteration times""",required=False)
parser.add_argument("--batch_size", type=int, default=16, help="Batch size.",required=False)
parser.add_argument("--batch_size_add", type=int, default=4, help="Batch size add.",required=False)
parser.add_argument("--max_epochs", type=int, default=100, help="Max epochs to train.",required=False)
parser.add_argument("--no_early_stop", action='store_false', dest='early_stop',
                    help="Disable early stop, which is based on sentence level accuracy.",required=False)
parser.add_argument("--patience", type=int, default=15, help="Patience to wait before stop.",required=False)
parser.add_argument("--learning_rate_decay", type=str, default='1', help="learning_rate_decay",required=False)
parser.add_argument("--learning_rate", type=float, default=0.001, help="The initial learning rate.",required=False)
parser.add_argument("--decay_steps", type=int, default=280 * 4, help="decay_steps.",required=False)
parser.add_argument("--decay_rate", type=float, default=0.9, help="decay_rate.",required=False)
parser.add_argument("--dataset", type=str, default='snips', help="""Type 'atis' or 'snips' to use dataset provided by us or enter what ever you named your own dataset.
                Note, if you don't want to use this part, enter --dataset=''. It can not be None""",required=False)
parser.add_argument("--model_path", type=str, default='./model', help="Path to save model.",required=False)
parser.add_argument("--vocab_path", type=str, default='./vocab', help="Path to vocabulary files.",required=False)
parser.add_argument("--train_data_path", type=str, default='train', help="Path to training data files.",required=False)
parser.add_argument("--test_data_path", type=str, default='test', help="Path to testing data files.",required=False)
parser.add_argument("--valid_data_path", type=str, default='valid', help="Path to validation data files.",required=False)
parser.add_argument("--input_file", type=str, default='seq.in', help="Input file name.",required=False)
parser.add_argument("--slot_file", type=str, default='seq.out', help="Slot file name.",required=False)
parser.add_argument("--intent_file", type=str, default='label', help="Intent file name.",required=False)
parser.add_argument("--embedding_path", type=str, default='', help="embedding array's path.",required=False)

arg = parser.parse_args(''.split())
if arg.dataset=='atis':
    arg.model_type='intent_only'
else:
    arg.model_type='full'

for k, v in sorted(vars(arg).items()):
    print(k, '=', v)
print()

if arg.model_type == 'full':
    remove_slot_attn = False 
elif arg.model_type == 'intent_only':
    remove_slot_attn = True
else:
    print('unknown model type!')
    exit(1)

if arg.dataset == None:
    print('name of dataset can not be None')
    exit(1)
elif arg.dataset == 'snips':
    print('use snips dataset')
elif arg.dataset == 'atis':
    print('use atis dataset')
else:
    print('use own dataset: ', arg.dataset)
full_train_path = os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/data', arg.dataset, arg.train_data_path)
full_test_path = os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/data', arg.dataset, arg.test_data_path)
full_valid_path = os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/data', arg.dataset, arg.valid_data_path)

createVocabulary(os.path.join(full_train_path, arg.input_file), os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'in_vocab'))
createVocabulary(os.path.join(full_train_path, arg.slot_file), os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'slot_vocab'))
createVocabulary(os.path.join(full_train_path, arg.intent_file), os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'intent_vocab'))

in_vocab = loadVocabulary(os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'in_vocab'))
slot_vocab = loadVocabulary(os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'slot_vocab'))
intent_vocab = loadVocabulary(os.path.join('/content/drive/My Drive/SF-ID-Network-For-NLU/Vocab', 'intent_vocab'))


def createModel(input_data, input_size, sequence_length, slots, slot_size, intent_size, layer_size=128,
                isTraining=True):
    cell_fw = tf.contrib.rnn.BasicLSTMCell(layer_size)
    cell_bw = tf.contrib.rnn.BasicLSTMCell(layer_size)

    if isTraining == True:
        cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, input_keep_prob=0.5,
                                                output_keep_prob=0.5)
        cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, input_keep_prob=0.5,
                                                output_keep_prob=0.5)
    if arg.embedding_path:
        embedding_weight = np.load(arg.embedding_path)
        embedding = tf.Variable(embedding_weight, name='embedding', dtype=tf.float32)
    else:
        embedding = tf.get_variable('embedding', [input_size, layer_size])
    inputs = tf.nn.embedding_lookup(embedding, input_data)
    state_outputs, final_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, inputs,
                                                                 sequence_length=sequence_length, dtype=tf.float32)
    final_state = tf.concat([final_state[0][0], final_state[0][1], final_state[1][0], final_state[1][1]], 1)
    state_outputs = tf.concat([state_outputs[0], state_outputs[1]], 2)
    state_shape = state_outputs.get_shape()

    with tf.variable_scope('attention'):
        slot_inputs = state_outputs
        if not remove_slot_attn:
            with tf.variable_scope('slot_attn'):
                attn_size = state_shape[2].value
                origin_shape = tf.shape(state_outputs)
                hidden = tf.expand_dims(state_outputs, 1)
                hidden_conv = tf.expand_dims(state_outputs, 2)
                k = tf.get_variable("AttnW", [1, 1, attn_size, attn_size])
                hidden_features = tf.nn.conv2d(hidden_conv, k, [1, 1, 1, 1], "SAME")
                hidden_features = tf.reshape(hidden_features, origin_shape)
                hidden_features = tf.expand_dims(hidden_features, 1)
                v = tf.get_variable("AttnV", [attn_size])
                slot_inputs_shape = tf.shape(slot_inputs)
                slot_inputs = tf.reshape(slot_inputs, [-1, attn_size])
                y = core_rnn_cell._linear(slot_inputs, attn_size, True)
                y = tf.reshape(y, slot_inputs_shape)
                y = tf.expand_dims(y, 2)
                s = tf.reduce_sum(v * tf.tanh(hidden_features + y), [3])
                a = tf.nn.softmax(s)
                a = tf.expand_dims(a, -1)
                slot_d = tf.reduce_sum(a * hidden, [2])
                slot_reinforce_state = tf.expand_dims(slot_d, 2)
        else:
            attn_size = state_shape[2].value
            slot_d=slot_inputs
            slot_reinforce_state = tf.expand_dims(slot_inputs, 2)
            slot_inputs = tf.reshape(slot_inputs, [-1, attn_size])

        intent_input = final_state
        with tf.variable_scope('intent_attn'):
            attn_size = state_shape[2].value
            hidden = tf.expand_dims(state_outputs, 2)
            k = tf.get_variable("AttnW", [1, 1, attn_size, attn_size])
            hidden_features = tf.nn.conv2d(hidden, k, [1, 1, 1, 1], "SAME")
            v = tf.get_variable("AttnV", [attn_size])

            y = core_rnn_cell._linear(intent_input, attn_size, True)
            y = tf.reshape(y, [-1, 1, 1, attn_size])
            s = tf.reduce_sum(v * tf.tanh(hidden_features + y), [2, 3])
            a = tf.nn.softmax(s)
            a = tf.expand_dims(a, -1)
            a = tf.expand_dims(a, -1)
            d = tf.reduce_sum(a * hidden, [1, 2]) 
            r_intent = d
            intent_context_states = d

        if arg.priority_order == 'intent_first':
            for n in range(arg.iteration_num):
                with tf.variable_scope('intent_subnet' + str(n - 1)):
                    attn_size = state_shape[2].value
                    hidden = tf.expand_dims(state_outputs, 2)
                    k1 = tf.get_variable("W1", [1, 1, attn_size, attn_size])
                    k2 = tf.get_variable('W2', [1, 1, attn_size, attn_size])
                    slot_reinforce_features = tf.nn.conv2d(slot_reinforce_state, k1, [1, 1, 1, 1],
                                                           "SAME")
                    hidden_features = tf.nn.conv2d(hidden, k2, [1, 1, 1, 1], "SAME")
                    v1 = tf.get_variable("AttnV", [attn_size])
                    bias = tf.get_variable("Bias", [attn_size])
                    s = tf.reduce_sum(v1 * tf.tanh(hidden_features + slot_reinforce_features + bias), [2, 3])
                    a = tf.nn.softmax(s)
                    a = tf.expand_dims(a, -1)
                    a = tf.expand_dims(a, -1)
                    r = tf.reduce_sum(a * slot_reinforce_state, [1, 2])

                    r_intent = r + intent_context_states

                    intent_output = tf.concat([r_intent, intent_input], 1)

                with tf.variable_scope('slot_subnet' + str(n - 1)):
                    intent_gate = core_rnn_cell._linear(r_intent, attn_size, True)
                    intent_gate = tf.reshape(intent_gate, [-1, 1, intent_gate.get_shape()[
                        1].value])
                    v1 = tf.get_variable("gateV", [attn_size])
                    relation_factor = v1 * tf.tanh(slot_d + intent_gate)
                    relation_factor = tf.reduce_sum(relation_factor, [2])
                    relation_factor = tf.expand_dims(relation_factor, -1)
                    slot_reinforce_state1 = slot_d * relation_factor
                    slot_reinforce_state = tf.expand_dims(slot_reinforce_state1, 2)
                    slot_reinforce_vector = tf.reshape(slot_reinforce_state1, [-1, attn_size])
                    slot_output = tf.concat([slot_reinforce_vector, slot_inputs], 1)

        else:
            for n in range(arg.iteration_num):
                with tf.variable_scope('slot_subnet' + str(n - 1)):
                    intent_gate = core_rnn_cell._linear(r_intent, attn_size, True)
                    intent_gate = tf.reshape(intent_gate, [-1, 1, intent_gate.get_shape()[
                        1].value])
                    v1 = tf.get_variable("gateV", [attn_size])
                    relation_factor = v1 * tf.tanh(slot_d + intent_gate)
                    relation_factor = tf.reduce_sum(relation_factor, [2])
                    relation_factor = tf.expand_dims(relation_factor, -1)
                    slot_reinforce_state = slot_d * relation_factor
                    slot_reinforce_vector = tf.reshape(slot_reinforce_state, [-1, attn_size])
                    slot_output = tf.concat([slot_reinforce_vector, slot_inputs], 1)

                with tf.variable_scope('intent_subnet' + str(n - 1)):
                    attn_size = state_shape[2].value
                    hidden = tf.expand_dims(state_outputs, 2)
                    slot_reinforce_output = tf.expand_dims(slot_reinforce_state, 2)
                    k1 = tf.get_variable("W1", [1, 1, attn_size, attn_size])
                    k2 = tf.get_variable('W2', [1, 1, attn_size, attn_size])
                    slot_features = tf.nn.conv2d(slot_reinforce_output, k1, [1, 1, 1, 1], "SAME")
                    hidden_features = tf.nn.conv2d(hidden, k2, [1, 1, 1, 1], "SAME")
                    v1 = tf.get_variable("AttnV", [attn_size])
                    bias = tf.get_variable("Bias", [attn_size])
                    s = tf.reduce_sum(v1 * tf.tanh(hidden_features + slot_features + bias), [2, 3])
                    a = tf.nn.softmax(s)
                    a = tf.expand_dims(a, -1)
                    a = tf.expand_dims(a, -1)
                    r = tf.reduce_sum(a * slot_reinforce_output, [1, 2])

                    r_intent = r + intent_context_states

                    intent_output = tf.concat([r_intent, intent_input], 1)

    with tf.variable_scope('intent_proj'):
        intent = core_rnn_cell._linear(intent_output, intent_size, True)
    with tf.variable_scope('slot_proj'):
        slot = core_rnn_cell._linear(slot_output, slot_size, True)
        if arg.use_crf:
            nstep = tf.shape(state_outputs)[1]
            slot = tf.reshape(slot, [-1, nstep, slot_size])
    outputs = [slot, intent]
    return outputs


input_data = tf.placeholder(tf.int32, [None, None], name='inputs')
sequence_length = tf.placeholder(tf.int32, [None], name="sequence_length")
global_step = tf.Variable(0, trainable=False, name='global_step')
slots = tf.placeholder(tf.int32, [None, None], name='slots')
slot_weights = tf.placeholder(tf.float32, [None, None], name='slot_weights')
intent = tf.placeholder(tf.int32, [None], name='intent')

with tf.variable_scope('model'):
    training_outputs = createModel(input_data, len(in_vocab['vocab']), sequence_length, slots, len(slot_vocab['vocab']),
                                   len(intent_vocab['vocab']), layer_size=arg.layer_size)

slots_shape = tf.shape(slots)
slots_reshape = tf.reshape(slots, [-1])

slot_outputs = training_outputs[0]
with tf.variable_scope('slot_loss'):
    if arg.use_crf:
        log_likelihood, trans_params = tf.contrib.crf.crf_log_likelihood(slot_outputs, slots, sequence_length)
        slot_loss = tf.reduce_mean(-log_likelihood)
    else:
        crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=slots_reshape, logits=slot_outputs)
        crossent = tf.reshape(crossent, slots_shape)
        slot_loss = tf.reduce_sum(crossent * slot_weights, 1)
        total_size = tf.reduce_sum(slot_weights, 1)
        total_size += 1e-12
        slot_loss = slot_loss / total_size

intent_output = training_outputs[1]
with tf.variable_scope('intent_loss'):
    crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=intent, logits=intent_output)
    intent_loss = tf.reduce_sum(crossent) / tf.cast(arg.batch_size, tf.float32)
params = tf.trainable_variables()
learning_rate = tf.train.exponential_decay(arg.learning_rate, global_step, arg.decay_steps, arg.decay_rate,
                                           staircase=False)
if arg.learning_rate_decay:
    opt = tf.train.AdamOptimizer(learning_rate)
else:
    opt = tf.train.AdamOptimizer(arg.learning_rate)
intent_params = []
slot_params = []
for p in params:
    if not 'slot_' in p.name:
        intent_params.append(p)
    if 'slot_' in p.name or 'bidirectional_rnn' in p.name or 'embedding' in p.name:
        slot_params.append(p)

gradients_slot = tf.gradients(slot_loss, slot_params)
gradients_intent = tf.gradients(intent_loss, intent_params)

clipped_gradients_slot, norm_slot = tf.clip_by_global_norm(gradients_slot, 5.0)
clipped_gradients_intent, norm_intent = tf.clip_by_global_norm(gradients_intent, 5.0)

gradient_norm_slot = norm_slot
gradient_norm_intent = norm_intent
update_slot = opt.apply_gradients(zip(clipped_gradients_slot, slot_params))
update_intent = opt.apply_gradients(zip(clipped_gradients_intent, intent_params), global_step=global_step)

training_outputs = [global_step, slot_loss, update_intent, update_slot, gradient_norm_intent, gradient_norm_slot]
inputs = [input_data, sequence_length, slots, slot_weights, intent]


with tf.variable_scope('model', reuse=True):
    inference_outputs = createModel(input_data, len(in_vocab['vocab']), sequence_length, slots,
                                    len(slot_vocab['vocab']),
                                    len(intent_vocab['vocab']), layer_size=arg.layer_size, isTraining=False)

if arg.use_crf:
    inference_slot_output, pred_scores = tf.contrib.crf.crf_decode(inference_outputs[0], trans_params, sequence_length)
else:
    inference_slot_output = tf.nn.softmax(inference_outputs[0], name='slot_output')

inference_intent_output = tf.nn.softmax(inference_outputs[1], name='intent_output')

inference_outputs = [inference_intent_output, inference_slot_output]
inference_inputs = [input_data, sequence_length]

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

saver = tf.train.Saver()
gpu_options = tf.GPUOptions(allow_growth=True)

with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    sess.run(tf.global_variables_initializer())
    logging.info('Training Start')

    epochs = 0
    loss = 0.0
    data_processor = None
    line = 0
    num_loss = 0
    step = 0
    no_improve = 0

    valid_slot = 0
    test_slot = 0
    valid_intent = 0
    test_intent = 0
    valid_err = 0
    test_err = 0
    best_epoch_num = 0
    while True:
        if data_processor == None:
            data_processor = DataProcessor(os.path.join(full_train_path, arg.input_file),
                                           os.path.join(full_train_path, arg.slot_file),
                                           os.path.join(full_train_path, arg.intent_file), in_vocab, slot_vocab,
                                           intent_vocab)
        in_data, slot_data, slot_weight, length, intents, _, _, _ = data_processor.get_batch(arg.batch_size)
        feed_dict = {input_data.name: in_data, slots.name: slot_data, slot_weights.name: slot_weight,
                     sequence_length.name: length, intent.name: intents}
        ret = sess.run(training_outputs, feed_dict)
        loss += np.mean(ret[1])

        line += arg.batch_size
        step = ret[0]
        num_loss += 1

        if data_processor.end == 1:
            arg.batch_size += arg.batch_size_add
            line = 0
            #data_processor.close()
            data_processor = None
            epochs += 1
            logging.info('Step: ' + str(step))
            logging.info('Epochs: ' + str(epochs))
            logging.info('Loss: ' + str(loss / num_loss))
            num_loss = 0
            loss = 0.0

            save_path = os.path.join(arg.model_path, '_step_' + str(step) + '_epochs_' + str(epochs) + '.ckpt')
            saver.save(sess, save_path)


            def valid(in_path, slot_path, intent_path):
                data_processor_valid = DataProcessor(in_path, slot_path, intent_path, in_vocab, slot_vocab,
                                                     intent_vocab)

                pred_intents = []
                correct_intents = []
                slot_outputs = []
                correct_slots = []
                input_words = []

                gate_seq = []
                while True:
                    in_data, slot_data, slot_weight, length, intents, in_seq, slot_seq, intent_seq = data_processor_valid.get_batch(
                        arg.batch_size)
                    if len(in_data) <= 0:
                        break
                    feed_dict = {input_data.name: in_data, sequence_length.name: length}
                    ret = sess.run(inference_outputs, feed_dict)
                    for i in ret[0]:
                        pred_intents.append(np.argmax(i))
                    for i in intents:
                        correct_intents.append(i)

                    pred_slots = ret[1].reshape((slot_data.shape[0], slot_data.shape[1], -1))
                    for p, t, i, l in zip(pred_slots, slot_data, in_data, length):
                        if arg.use_crf:
                            p = p.reshape([-1])
                        else:
                            p = np.argmax(p, 1)
                        tmp_pred = []
                        tmp_correct = []
                        tmp_input = []
                        for j in range(l):
                            tmp_pred.append(slot_vocab['rev'][p[j]])
                            tmp_correct.append(slot_vocab['rev'][t[j]])
                            tmp_input.append(in_vocab['rev'][i[j]])

                        slot_outputs.append(tmp_pred)
                        correct_slots.append(tmp_correct)
                        input_words.append(tmp_input)

                    if data_processor_valid.end == 1:
                        break

                pred_intents = np.array(pred_intents)
                correct_intents = np.array(correct_intents)
                accuracy = (pred_intents == correct_intents)
                semantic_acc = accuracy
                accuracy = accuracy.astype(float)
                accuracy = np.mean(accuracy) * 100.0

                index = 0
                for t, p in zip(correct_slots, slot_outputs):
                    # Process Semantic Error
                    if len(t) != len(p):
                        raise ValueError('Error!!')

                    for j in range(len(t)):
                        if p[j] != t[j]:
                            semantic_acc[index] = False
                            break
                    index += 1
                semantic_acc = semantic_acc.astype(float)
                semantic_acc = np.mean(semantic_acc) * 100.0

                f1, precision, recall = computeF1Score(correct_slots, slot_outputs)
                logging.info('slot f1: ' + str(f1))
                logging.info('intent accuracy: ' + str(accuracy))
                logging.info('semantic Acc(intent, slots are all correct): ' + str(semantic_acc))

                #data_processor_valid.close()
                return f1, accuracy, semantic_acc, pred_intents, correct_intents, slot_outputs, correct_slots, input_words, gate_seq


            logging.info('Valid:')
            epoch_valid_slot, epoch_valid_intent, epoch_valid_err, valid_pred_intent, valid_correct_intent, valid_pred_slot, valid_correct_slot, valid_words, valid_gate = valid(
                os.path.join(full_valid_path, arg.input_file), os.path.join(full_valid_path, arg.slot_file),
                os.path.join(full_valid_path, arg.intent_file))

            logging.info('Test:')
            epoch_test_slot, epoch_test_intent, epoch_test_err, test_pred_intent, test_correct_intent, test_pred_slot, test_correct_slot, test_words, test_gate = valid(
                os.path.join(full_test_path, arg.input_file), os.path.join(full_test_path, arg.slot_file),
                os.path.join(full_test_path, arg.intent_file))

            if epoch_test_err <= test_err:
                no_improve += 1
            else:
                best_epoch_num = epochs
                test_err = epoch_test_err

                no_improve = 0

            if test_err > 0:
                logging.info('best epoch_num :  {}'.format(best_epoch_num))
                logging.info('best score : {}'.format(test_err))

            if epochs == arg.max_epochs:
                break

            if arg.early_stop == True:
                if no_improve > arg.patience:
                    break

batch_size = 16
batch_size_add = 4
cell = lstm
dataset = snips
decay_rate = 0.9
decay_steps = 1120
early_stop = True
embedding_path = 
input_file = seq.in
intent_file = label
iteration_num = 1
layer_size = 64
learning_rate = 0.001
learning_rate_decay = 1
max_epochs = 100
model_path = ./model
model_type = full
patience = 15
priority_order = slot_first
slot_file = seq.out
test_data_path = test
train_data_path = train
use_crf = False
use_embedding = 1
valid_data_path = valid
vocab_path = ./vocab

use snips dataset
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensor

2020-07-26 08:26:13,558 : INFO : Training Start
2020-07-26 08:26:56,911 : INFO : Step: 818
2020-07-26 08:26:56,912 : INFO : Epochs: 1
2020-07-26 08:26:56,915 : INFO : Loss: 1.4120219957201872
2020-07-26 08:26:57,293 : INFO : Valid:
2020-07-26 08:26:57,824 : INFO : slot f1: 57.92666842521762
2020-07-26 08:26:57,824 : INFO : intent accuracy: 98.42857142857143
2020-07-26 08:26:57,826 : INFO : semantic Acc(intent, slots are all correct): 31.142857142857146
2020-07-26 08:26:57,828 : INFO : Test:
2020-07-26 08:26:58,233 : INFO : slot f1: 55.25149856658848
2020-07-26 08:26:58,234 : INFO : intent accuracy: 95.85714285714285
2020-07-26 08:26:58,235 : INFO : semantic Acc(intent, slots are all correct): 30.28571428571429
2020-07-26 08:26:58,236 : INFO : best epoch_num :  1
2020-07-26 08:26:58,238 : INFO : best score : 30.28571428571429
2020-07-26 08:27:36,902 : INFO : Step: 1473
2020-07-26 08:27:36,903 : INFO : Epochs: 2
2020-07-26 08:27:36,904 : INFO : Loss: 0.547967822428878
2020-07-26 08:27:37

Instructions for updating:
Use standard file APIs to delete files with this prefix.


2020-07-26 08:29:58,522 : WARNING : From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/training/saver.py:963: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
2020-07-26 08:29:58,793 : INFO : Valid:
2020-07-26 08:29:59,151 : INFO : slot f1: 86.6324601674318
2020-07-26 08:29:59,152 : INFO : intent accuracy: 98.85714285714286
2020-07-26 08:29:59,154 : INFO : semantic Acc(intent, slots are all correct): 70.71428571428572
2020-07-26 08:29:59,155 : INFO : Test:
2020-07-26 08:29:59,714 : INFO : slot f1: 82.93597642646664
2020-07-26 08:29:59,715 : INFO : intent accuracy: 97.14285714285714
2020-07-26 08:29:59,718 : INFO : semantic Acc(intent, slots are all correct): 66.14285714285715
2020-07-26 08:29:59,721 : INFO : best epoch_num :  6
2020-07-26 08:29:59,722 : INFO : best score : 66.14285714285715
2020-07-26 08:30:3

In [ ]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
print ('Confusion Matrix :')
results = confusion_matrix(test_correct_intent, test_pred_intent) 
print(results) 
print ('Accuracy Score :',accuracy_score(test_correct_intent, test_pred_intent) )
print ('Report : ')
print (classification_report(test_correct_intent, test_pred_intent) )

Confusion Matrix :
[[ 83   0   0   0   0   2   1]
 [  0 102   2   0   0   0   0]
 [  0   0  92   0   0   0   0]
 [  0   0   0  80   0   0   0]
 [  0   0   0   1 100   6   0]
 [  4   0   0   0   4  99   0]
 [  0   0   0   0   0   0 124]]
Accuracy Score : 0.9714285714285714
Report : 
              precision    recall  f1-score   support

           0       0.95      0.97      0.96        86
           1       1.00      0.98      0.99       104
           2       0.98      1.00      0.99        92
           3       0.99      1.00      0.99        80
           4       0.96      0.93      0.95       107
           5       0.93      0.93      0.93       107
           6       0.99      1.00      1.00       124

    accuracy                           0.97       700
   macro avg       0.97      0.97      0.97       700
weighted avg       0.97      0.97      0.97       700



In [ ]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
print ('Confusion Matrix :')
results = confusion_matrix(valid_correct_intent, valid_pred_intent) 
print(results) 
print ('Accuracy Score :',accuracy_score(valid_correct_intent, valid_pred_intent) )
print ('Report : ')
print (classification_report(valid_correct_intent, valid_pred_intent) )

Confusion Matrix :
[[ 98   0   0   0   0   0   2]
 [  0  99   0   0   1   0   0]
 [  0   0 100   0   0   0   0]
 [  0   0   0 100   0   0   0]
 [  0   0   0   0  98   2   0]
 [  0   0   0   0   0 100   0]
 [  0   0   0   0   0   0 100]]
Accuracy Score : 0.9928571428571429
Report : 
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       100
           1       1.00      0.99      0.99       100
           2       1.00      1.00      1.00       100
           3       1.00      1.00      1.00       100
           4       0.99      0.98      0.98       100
           5       0.98      1.00      0.99       100
           6       0.98      1.00      0.99       100

    accuracy                           0.99       700
   macro avg       0.99      0.99      0.99       700
weighted avg       0.99      0.99      0.99       700



In [ ]:
print(test_pred_intent[0], test_correct_intent[0], test_pred_slot[0], test_correct_slot[0],sep='\n')

3
3
['O', 'B-object_select', 'B-object_type', 'B-rating_value', 'O', 'O', 'B-best_rating']
['O', 'B-object_select', 'B-object_type', 'B-rating_value', 'O', 'O', 'B-best_rating']


In [ ]:
for i in range(20):
  print(test_pred_intent[i], test_correct_intent[i], test_pred_slot[i], test_correct_slot[i],sep='\n')

3
3
['O', 'B-object_select', 'B-object_type', 'B-rating_value', 'O', 'O', 'B-best_rating']
['O', 'B-object_select', 'B-object_type', 'B-rating_value', 'O', 'O', 'B-best_rating']
6
6
['O', 'B-entity_name', 'I-entity_name', 'O', 'B-playlist_owner', 'B-playlist', 'I-playlist', 'I-playlist', 'I-playlist', 'O']
['O', 'B-artist', 'I-artist', 'O', 'B-playlist_owner', 'B-playlist', 'I-playlist', 'I-playlist', 'I-playlist', 'O']
3
3
['O', 'B-object_select', 'B-object_type', 'B-rating_value', 'B-rating_unit', 'O', 'O', 'B-best_rating']
['O', 'B-object_select', 'B-object_type', 'B-rating_value', 'B-rating_unit', 'O', 'O', 'B-best_rating']
5
5
['O', 'B-object_name', 'I-object_name', 'I-object_name', 'I-object_name']
['O', 'B-object_name', 'I-object_name', 'I-object_name', 'I-object_name']
5
5
['O', 'O', 'B-object_type', 'I-object_type', 'O', 'B-object_name', 'I-object_name']
['O', 'O', 'B-object_type', 'I-object_type', 'O', 'B-object_name', 'I-object_name']
6
6
['O', 'O', 'B-playlist_owner', 'O', 